In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from textwrap import wrap

import emoji
import joblib
import langdetect
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

plt.style.use("fivethirtyeight")
warnings.filterwarnings("ignore")

In [ ]:
def get_category_dict(category_file):
  category = pd.read_json(category_file, orient="records")
  category = pd.DataFrame(category["items"].values.tolist())

  return {
      cat.id: cat.snippet.get("title")
      for cat in category.itertuples(index=False)
  }

In [ ]:
def get_category_dict(category_file):
  category = pd.read_json(category_file, orient="records")
  category = pd.DataFrame(category["items"].values.tolist())

  return {
      cat.id: cat.snippet.get("title")
      for cat in category.itertuples(index=False)
  }

In [ ]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from textwrap import wrap

import emoji
import joblib
import langdetect
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

plt.style.use("fivethirtyeight")
warnings.filterwarnings("ignore")

In [ ]:
def get_category_dict(category_file):
  category = pd.read_json(category_file, orient="records")
  category = pd.DataFrame(category["items"].values.tolist())

  return {
      cat.id: cat.snippet.get("title")
      for cat in category.itertuples(index=False)
  }

In [ ]:
def get_category_dict(category_file):
  category = pd.read_json(category_file, orient="records")
  category = pd.DataFrame(category["items"].values.tolist())

  return {
      cat.id: cat.snippet.get("title")
      for cat in category.itertuples(index=False)
  }

In [ ]:
category_dict = get_category_dict('/content/sample_data/category.json')

In [ ]:
trending = pd.read_csv("/content/sample_data/trending.csv", parse_dates=["publish_time", "trending_time"])
with pd.option_context("display.max_columns", None):
  display(trending.head())

In [ ]:
start_date = trending.trending_time.min()
end_date = trending.trending_time.max()

print("start_date = ", start_date)
print("start_date = ", start_date)

start_date =  2021-02-07 05:46:51.832614+00:00
start_date =  2021-02-07 05:46:51.832614+00:00


In [ ]:
#delete data dari bulan februari - juni 

filtered_trending = trending[trending.trending_time.dt.month >= 7]

start_date = filtered_trending.trending_time.min()
end_date = filtered_trending.trending_time.max()

print("start_date = ", start_date)
print("start_date = ", start_date)

In [ ]:
num_videos = filtered_trending.shape[0]
print(f'{num_videos :}')

In [ ]:
# Distribusi Missing Value
filtered_trending.info()

In [ ]:
# Delete video dengan description kosong
filtered_trending.dropna(subset=["description"], inplace=True)

In [ ]:
trending_by_date = filtered_trending.groupby(
 filtered_trending.trending_time.dt.date
)
num_trending_per_day = trending_by_date.trending_time.count()
print("Number of videos in trending per day:", num_trending_per_day.unique())

In [ ]:
trending_duration = filtered_trending.groupby("title").agg(
 trending_duration=pd.NamedAgg(column="trending_time", aggfunc="count"),
 trending_start_date=pd.NamedAgg(column="trending_time", aggfunc="min"),
 trending_last_date=pd.NamedAgg(column="trending_time", aggfunc="max")
).sort_values("trending_duration", ascending=False).reset_index()
trending_duration.head(10)

In [ ]:
# Visualisasi data trending dengan chart bar
plt.figure(figsize=(15, 6))
plt.bar(
 trending_duration.title[:10].apply(lambda title: "\n".join(wrap(title, width=10))),
 trending_duration.trending_duration[:10]
)
plt.title("Longest Duration of Videos included in YouTube Trending Video", loc="left")
plt.xlabel("Video Title")
plt.ylabel("Trending Duration (in days)")
plt.grid(False)
plt.show()

In [ ]:
# Visualisasi data trending dengan chart bar
plt.figure(figsize=(15, 6))
plt.bar(
 trending_duration.title[:10].apply(lambda title: "\n".join(wrap(title, width=10))),
 trending_duration.trending_duration[:10]
)
plt.title("Longest Duration of Videos included in YouTube Trending Video", loc="left")
plt.xlabel("Video Title")
plt.ylabel("Trending Duration (in days)")
plt.grid(False)
plt.show()

In [ ]:
trending_by_title = filtered_trending.groupby("title")
trending_rewind = trending_by_title[["view", "like", "dislike"]].agg(["min", "max", "mean", "sum"])
trending_rewind

In [ ]:
# Visualisasi Trending video teratas masing2
top_10_liked = trending_rewind["like"].sort_values("max", ascending=False).iloc[:10]
plt.figure(figsize=(15, 6))
plt.bar(
 top_10_liked.index.to_series().apply(lambda title: "\n".join(wrap(title, width=10))),
 top_10_liked["max"],
 label="last like"
)
plt.bar(
 top_10_liked.index.to_series().apply(lambda title: "\n".join(wrap(title, width=10))),
 top_10_liked["min"],
 label="start like"
)
plt.title("Most videos in trending list improves drastically in terms of likes", loc="left", y=1.1)
plt.xlabel("Number of Like")
plt.ylabel("Video Title")
plt.legend()
plt.grid(False)
plt.show()

In [ ]:
top_10_viewed = trending_rewind["view"].sort_values("max", ascending=False).iloc[:10]

plt.figure(figsize=(15, 6))
plt.bar(
    top_10_viewed.index.to_series().apply(lambda title: "\n".join(wrap(title, width=10))),
    top_10_viewed["max"],
    label="last views"
)
plt.bar(
    top_10_viewed.index.to_series().apply(lambda title: "\n".join(wrap(title, width=10))),
    top_10_viewed["min"],
    label="last views"
)
plt.title("Most videos in trending list improve drastically in terms of views", loc="left", y=1.1)
plt.xlabel("Number of Views")
plt.ylabel("Video Title")
plt.legend()
plt.grid(False)
plt.show()

In [ ]:
top_10_disliked = trending_rewind["dislike"].sort_values("max", ascending=False).iloc[:10]
plt.figure(figsize=(15, 6))
plt.bar(
 top_10_disliked.index.to_series().apply(lambda title: "\n".join(wrap(title, width=10))),
 top_10_disliked["max"],
 label="last dislike"
)
plt.bar(
 top_10_disliked.index.to_series().apply(lambda title: "\n".join(wrap(title, width=10))),
 top_10_disliked["min"],
 label="start dislike"
)
plt.title("Most disliked videos in trending list are of shorts and gain more dislikes", loc="left", y=1.1)
plt.ylabel("Number of Dislike")
plt.xlabel("Video Title")
plt.legend()
plt.grid(False)
plt.show()


In [ ]:
# Sebagai contoh, perhatikan sampel judul dan deskripsi video berikut.
sample = filtered_trending.sample(10, random_state=11)
sample[["title", "description"]]

In [ ]:
# Merubah title dan descripsi video menjadi lowercase
sample["title_lang"] = sample.title.apply(lambda title: langdetect.detect(title.lower()))
sample["desc_lang"] = sample.description.apply(lambda desc: langdetect.detect(desc.lower()))

In [ ]:
with pd.option_context("display.max_colwidth", 100):
 display(sample[["title", "title_lang", "description", "desc_lang"]])

In [ ]:
def detect_language(text):
 """Detect language of the `text`."""
 try: 
  lang = langdetect.detect(text)
  return lang
 except:
  return

In [ ]:
filtered_trending["title_lang"] = filtered_trending["title"].apply(detect_language)
filtered_trending["desc_lang"] = filtered_trending["description"].apply(detect_language)

In [ ]:
filtered_trending[["title", "title_lang", "description", "desc_lang"]]


In [ ]:
indo_trending = filtered_trending.loc[
 (filtered_trending.title_lang == "id") | (filtered_trending.desc_lang == "id")
]
with pd.option_context("display.max_columns", None):
 display(indo_trending.sample(10))

In [ ]:
data = indo_trending[["title", "description", "category_id"]].reset_index(drop=True)
data

In [ ]:
data.drop_duplicates(subset="title", inplace=True)

In [ ]:
data.reset_index(drop=True, inplace=True)
data.shape

In [ ]:
list_emoji = [e for e in emoji.UNICODE_EMOJI.get("en")]

count = 0
for em in list_emoji:
  for title in data.title:
    if em in title:
      count += 1
print("How many titles use emoji?", count)

AttributeError: ignored

In [ ]:
def demojize(text):
 for em in list_emoji:
 if em in text:
 em_text = emoji.demojize(em)
 text = text.replace(em, " " + em_text + " ")
 return text


In [ ]:
def demojize(text):
 for em in list_emoji:
 if em in text:
 em_text = emoji.demojize(em)
 text = text.replace(em, " " + em_text + " ")
 return text

In [ ]:
data["title_emoji"] = data.title.apply(demojize)

In [ ]:
title_with_emoji_idx = [
 idx for idx in range(len(data.title))
 for em in list_emoji
 if em in data.loc[idx, "title"]
]

In [ ]:
with pd.option_context("display.max_colwidth", 100):
 display(data.loc[title_with_emoji_idx])

In [ ]:
# Delete kolom title
data.drop(columns="title", inplace=True)

In [ ]:
desc_with_emoji_idx = [
 idx for idx in range(len(data.description))
 for em in list_emoji
 if em in data.loc[idx, "description"]
]
data["desc_emoji"] = data.description.apply(demojize)
with pd.option_context("display.max_colwidth", 100):
 display(data.loc[desc_with_emoji_idx])

In [ ]:
data.drop(columns="description", inplace=True)

In [ ]:
data["all_text"] = data["title_emoji"] + " " + data["desc_emoji"]

In [ ]:
# data split
X_train, X_dev, y_train, y_dev = train_test_split(
 data["all_text"], data["category_id"],
 test_size=.2,
 stratify=data["category_id"],
 random_state=11
)
training_size = X_train.shape[0]
dev_size = X_dev.shape[0]
print(f"{training_size = }.. {dev_size = }")
# define vectorizer
vectorizer = TfidfVectorizer(
 min_df=.015,
 max_df=.7,
 ngram_range=(1, 1),
)
# generate tf-idf matrix
train_tfidf = vectorizer.fit_transform(X_train)
dev_tfidf = vectorizer.transform(X_dev)
print("Got train tf-idf with shape:", train_tfidf.shape)
print("Got dev tf-idf with shape:", dev_tfidf.shape)
# convert to dataframe
train_tfidf = pd.DataFrame(train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
dev_tfidf = pd.DataFrame(dev_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
with pd.option_context("display.max_columns", 100):
 display(train_tfidf.sample(5))

In [ ]:
dict_models = {
 "logistic_regression": LogisticRegression(),
 "naive_bayes": MultinomialNB(),
 "svm": LinearSVC(random_state=11),
 "decision_tree": DecisionTreeClassifier(random_state=11),
 "random_forest": RandomForestClassifier(random_state=11)
}


In [ ]:
for model in dict_models.values():
 print(f"-- {model.__class__.__name__} --")
 model.fit(train_tfidf, y_train)
 y_pred = model.predict(dev_tfidf)
 print("Reports on dev set:", classification_report(y_dev, y_pred), sep="\n")

In [ ]:
svm_grid_search = GridSearchCV(
 dict_models["svm"],
 {"C": (10, 1, .1, .05, .01)},
)
svm_grid_search.fit(train_tfidf, y_train)
svm_pred_dev = svm_grid_search.predict(dev_tfidf)
print("Reports on train set:",
 classification_report(
 y_train,
 svm_grid_search.predict(train_tfidf)
 ), sep="\n")
print("Reports on dev set:", classification_report(y_dev, svm_pred_dev), sep="\n")

In [ ]:
svm_grid_search.best_params_

In [ ]:
random_forest_grid_search = GridSearchCV(
 dict_models["random_forest"],
 {
 "n_estimators": (10, 20, 25, 50, 75, 100, 125),
 "max_depth": (5, 10, 25, 50),
 }
)
random_forest_grid_search.fit(train_tfidf, y_train)
random_forest_pred = random_forest_grid_search.predict(dev_tfidf)
print("Reports on train set:",
 classification_report(
 y_train,
 random_forest_grid_search.predict(train_tfidf)
 ),
 sep="\n"
)
print("Reports on dev set:", classification_report(y_dev, random_forest_pred), sep="\n")


In [ ]:
random_forest_grid_search.best_params_

In [ ]:
model = Pipeline([
 ("vectorizer", TfidfVectorizer(
 min_df=.015,
 max_df=.7,
 ngram_range=(1, 1),
 )),
 ("model", RandomForestClassifier(
 max_depth=50,
 n_estimators=75,
 random_state=11
 ))
])
# training
model.fit(X_train, y_train)
pred = model.predict(X_dev)
print("Reports on train set:",
 classification_report(
 y_train,
 model.predict(X_train)
 ),
 sep="\n"
)
print("Reports on dev set:", classification_report(y_dev, pred), sep="\n")

In [ ]:
joblib.dump(model, "D:/PTA 2016-2017/Modul/dataset/modelyt.joblib")

In [ ]:
model = joblib.load("D:/PTA 2016-2017/Modul/dataset/modelyt.joblib")
print(model.get_params())

In [ ]:
preds = model.predict(X_dev)
print(classification_report(y_dev, preds))